In [ ]:
import onnxruntime
import numpy as np
import torch
session = onnxruntime.InferenceSession("/data2/user/sjj995/TensorRT/test/ssd_mobilenet_v1_10.onnx",providers=['CPUExecutionProvider','CUDAExecutionProvider' ])
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
session

In [ ]:
session.get_modelmeta()


In [ ]:
[output.name for output in session.get_outputs()]

In [ ]:
for temp in session.get_inputs():
    print(temp)
    print(temp.type)

In [ ]:
for temp2 in session.get_inputs():
    print(temp2)

In [ ]:
session2 = onnxruntime.InferenceSession("/data2/user/sjj995/TensorRT/onnx_models/bidaf-9.onnx",providers=['CPUExecutionProvider','CUDAExecutionProvider' ])
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
for temp in session2.get_inputs():
    print(temp)

In [ ]:
import re
inputs={}
input_shape = []
type_catcher = re.compile('\w+\((\w+?)\)')

for temp in session2.get_inputs():
    print(temp.type)
    print(type_catcher.match(temp.type).group(1))
    print(temp.shape)
    input_shape.append(temp.shape)
    
inputs['shape'] = input_shape

In [ ]:
for input_s in inputs['shape']:
    print(input_s)
    if 'c' in input_s:
        print('hello')


In [ ]:
len(inputs['shape'])

In [ ]:
# yolo
for temp in session2.get_inputs():
    print(temp)

In [ ]:
session3 = onnxruntime.InferenceSession("/data2/user/sjj995/TensorRT/verified/unsupported/bertsquad-12.onnx",providers=['CPUExecutionProvider','CUDAExecutionProvider' ])
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
for temp in session3.get_inputs():
    print(temp)

In [ ]:
session4 = onnxruntime.InferenceSession("/data2/user/sjj995/TensorRT/verified/unsupported/tiny-yolov3-11.onnx",providers=['CPUExecutionProvider','CUDAExecutionProvider' ])
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
for temp in session4.get_inputs():
    print(temp)

In [ ]:
len(session.get_inputs()[0].shape)

In [ ]:
session.get_inputs()[1].shape

In [ ]:
session.get_overridable_initializers()

In [ ]:
session.get_inputs()[0]

In [ ]:
input_shape = session.get_inputs()[0].shape
input_shape

In [ ]:
for i in range(1,len(input_shape)):
    if type(input_shape[i]) == str:
        globals()['dim_{}'.format(i)] = 255
    else:
        globals()['dim_{}'.format(i)] = input_shape[i]

In [ ]:
dim_1

In [ ]:
tuple(input_shape)

In [ ]:
input_shape.explode

In [ ]:
type(session.get_inputs()[0].shape[0])

In [ ]:
session.get_inputs()[0].shape[0]

In [ ]:
session.get_inputs()[0].shape[1]

In [ ]:
session.get_inputs()[0].shape[2]

In [ ]:
session.get_inputs()[0].shape[3]

In [ ]:
session.get_outputs()[0].shape

In [ ]:
session.get_modelmeta()
first_input_name = session.get_inputs()[0].name
first_output_name = session.get_outputs()[0].name

In [ ]:
first_input_name

In [ ]:
first_output_name

In [ ]:
indata = torch.randn(1,3, 224, 224).to(device)

In [ ]:
indata
X=torch.randn(3, 224, 224)
print(X)

In [ ]:
# results = session.run(["output1", "output2"], {
#                       "input1": indata1, "input2": indata2})

results = session.run(None, {'input':to_numpy(indata)})
results

In [ ]:
import onnx


onnx_model = onnx.load("/datadrive/tensorrt/onnx_models/mobilenetv2-12.onnx")
onnx.checker.check_model(onnx_model)

In [ ]:
import torch
import onnxruntime
import numpy
#tiny-yolov3-11.onnx
ort_session = onnxruntime.InferenceSession("/data2/user/sjj995/TensorRT/onnx_models/yolov3-12.onnx",providers=['CPUExecutionProvider','CUDAExecutionProvider' ])


def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
x = torch.randn(1,3, 128,128).to(device)
x2 = torch.randn(1,2).to(device)
print(x2)
#x2 = torch.tensor([[460.,  840.]]).to(device)
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x),ort_session.get_inputs()[1].name: to_numpy(x2)}
outname = [output.name for output in ort_session.get_outputs()]
#print(ort_inputs)
#print(outname)
ort_outs = ort_session.run(outname,ort_inputs)

# compare ONNX Runtime and PyTorch results
# np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

#print("Exported model has been tested with ONNXRuntime, and the result looks good!")
#print((ort_outs))
print((ort_outs[0].shape))
print((ort_outs[1].shape))
print((ort_outs[2].shape))
print((ort_outs[2]))

In [ ]:
len(ort_outs)

In [ ]:
import time
import onnxruntime
import numpy as np
from PIL import Image

session = onnxruntime.InferenceSession("/data2/user/sjj995/TensorRT/onnx_models/yolov3-12.onnx")
inname = [input.name for input in session.get_inputs()]
outname = [output.name for output in session.get_outputs()]

def letterbox_image(image, size):
    iw, ih = image.size
    w, h = size
    scale = min(w/iw, h/ih)
    nw = int(iw*scale)
    nh = int(ih*scale)

    image = image.resize((nw,nh), Image.BICUBIC)
    new_image = Image.new('RGB', size, (128,128,128))
    new_image.paste(image, ((w-nw)//2, (h-nh)//2))
    return new_image

def preprocess(img):
    model_image_size = (416, 416)
    boxed_image = letterbox_image(img, tuple(reversed(model_image_size)))
    image_data = np.array(boxed_image, dtype='float32')
    image_data /= 255.
    image_data = np.transpose(image_data, [2, 0, 1])
    image_data = np.expand_dims(image_data, 0)
    return image_data

def get_prediction(image_data, image_size):
    input = {
        inname[0]: image_data,
        inname[1]: image_size
    }
    t0 = time.time()
    boxes, scores, indices = session.run(outname, input)
    print("Predict Time: %ss" % (time.time() - t0))
    out_boxes, out_scores, out_classes = [], [], []
    print(boxes.shape)
    print(indices)
    print(indices.shape)
    for idx_ in indices:
        out_classes.append(idx_[1])
        out_scores.append(scores[tuple(idx_)])
        idx_1 = (idx_[0], idx_[2])
        out_boxes.append(boxes[idx_1])
    return out_boxes, out_scores, out_classes

label =["person","bicycle","car","motorbike","aeroplane","bus","train","truck","boat",
    "traffic light","fire hydrant","stop sign","parking meter","bench","bird","cat",
    "dog","horse","sheep","cow","elephant","bear","zebra","giraffe","backpack","umbrella",
    "handbag","tie","suitcase","frisbee","skis","snowboard","sports ball","kite","baseball bat",
    "baseball glove","skateboard","surfboard","tennis racket","bottle","wine glass","cup","fork",
    "knife","spoon","bowl","banana","apple","sandwich","orange","broccoli","carrot","hot dog",
    "pizza","donut","cake","chair","sofa","pottedplant","bed","diningtable","toilet","tvmonitor",
    "laptop","mouse","remote","keyboard","cell phone","microwave","oven","toaster","sink",
    "refrigerator","book","clock","vase","scissors","teddy bear","hair drier","toothbrush"]

image = Image.open("/data2/user/sjj995/TensorRT/person.jpg")
image_data = preprocess(image)
image_size = np.array([image.size[1], image.size[0]], dtype=np.float32).reshape(1, 2)
out_boxes, out_scores, out_classes = get_prediction(image_data, image_size)
out_boxes = np.array(out_boxes).tolist()
out_scores = np.array(out_scores).tolist()
out_classes = np.array(out_classes).tolist()

for i, c in reversed(list(enumerate(out_classes))):
    print("box:", out_boxes[i])
    print("score:", out_scores[i],",", label[c])

BiDAF

In [ ]:
import numpy as np
import string
from nltk import word_tokenize

def preprocess(text):
   tokens = word_tokenize(text)
   # split into lower-case word tokens, in numpy array with shape of (seq, 1)
   words = np.asarray([w.lower() for w in tokens]).reshape(-1, 1)
   # split words into chars, in numpy array with shape of (seq, 1, 1, 16)
   chars = [[c for c in t][:16] for t in tokens]
   chars = [cs+['']*(16-len(cs)) for cs in chars]
   chars = np.asarray(chars).reshape(-1, 1, 1, 16)
   return words, chars

# input
context = 'A quick brown fox jumps over the lazy dog.'
query = 'What color is the fox?'
cw, cc = preprocess(context)
qw, qc = preprocess(query)

In [ ]:
tt='[uint8]'

In [ ]:
import re
ttt = re.sub('[\[\]\']','',tt)

In [ ]:
tttt = re.compile('[\[\]\']')

In [ ]:
re.sub(tttt,'',tt)

In [5]:
# import torch
# import onnxruntime
# import numpy
# ort_session = onnxruntime.InferenceSession("/data2/user/sjj995/TensorRT/temp2/vgg16-7.onnx",providers=['CPUExecutionProvider','CUDAExecutionProvider' ])


# def to_numpy(tensor):
#     return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# # compute ONNX Runtime output prediction
# x = torch.randn(1,3, 224,224).to(device)
# ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
# outname = [output.name for output in ort_session.get_outputs()]
# #print(ort_inputs)
# #print(outname)
# ort_outs = ort_session.run(outname,ort_inputs)

# # compare ONNX Runtime and PyTorch results
# # np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

# #print("Exported model has been tested with ONNXRuntime, and the result looks good!")
# print((ort_outs))

import onnx
model_file = "/data2/user/sjj995/TensorRT/temp2/vgg16-7.onnx"
onnx_model_detailed = onnx.load(model_file)
for node in onnx_model_detailed.graph.node:
    print(node.output)

['vgg0_conv0_fwd']
['vgg0_relu0_fwd']
['vgg0_conv1_fwd']
['vgg0_relu1_fwd']
['vgg0_pool0_fwd']
['vgg0_conv2_fwd']
['vgg0_relu2_fwd']
['vgg0_conv3_fwd']
['vgg0_relu3_fwd']
['vgg0_pool1_fwd']
['vgg0_conv4_fwd']
['vgg0_relu4_fwd']
['vgg0_conv5_fwd']
['vgg0_relu5_fwd']
['vgg0_conv6_fwd']
['vgg0_relu6_fwd']
['vgg0_pool2_fwd']
['vgg0_conv7_fwd']
['vgg0_relu7_fwd']
['vgg0_conv8_fwd']
['vgg0_relu8_fwd']
['vgg0_conv9_fwd']
['vgg0_relu9_fwd']
['vgg0_pool3_fwd']
['vgg0_conv10_fwd']
['vgg0_relu10_fwd']
['vgg0_conv11_fwd']
['vgg0_relu11_fwd']
['vgg0_conv12_fwd']
['vgg0_relu12_fwd']
['vgg0_pool4_fwd']
['flatten_60']
['vgg0_dense0_fwd']
['vgg0_dense0_relu_fwd']
['vgg0_dropout0_fwd']
['flatten_65']
['vgg0_dense1_fwd']
['vgg0_dense1_relu_fwd']
['vgg0_dropout1_fwd']
['flatten_70']
['vgg0_dense2_fwd']


In [15]:
print(type(onnx_model_detailed.graph))
for g in onnx_model_detailed.graph.node:
    print(g)

<class 'onnx.onnx_ml_pb2.GraphProto'>
input: "data"
input: "vgg0_conv0_weight"
input: "vgg0_conv0_bias"
output: "vgg0_conv0_fwd"
name: "vgg0_conv0_fwd"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "group"
  i: 1
  type: INT
}
attribute {
  name: "kernel_shape"
  ints: 3
  ints: 3
  type: INTS
}
attribute {
  name: "pads"
  ints: 1
  ints: 1
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}

input: "vgg0_conv0_fwd"
output: "vgg0_relu0_fwd"
name: "vgg0_relu0_fwd"
op_type: "Relu"

input: "vgg0_relu0_fwd"
input: "vgg0_conv1_weight"
input: "vgg0_conv1_bias"
output: "vgg0_conv1_fwd"
name: "vgg0_conv1_fwd"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "group"
  i: 1
  type: INT
}
attribute {
  name: "kernel_shape"
  ints: 3
  ints: 3
  type: INTS
}
attribute {
  name: "pads"
  ints: 1
  ints: 1
  ints: 1
  ints: 1
  type: INTS
}
a

In [ ]:
from onnx import shape_inference
inferred_model = shape_inference.infer_shapes(onnx_model_detailed)

In [10]:
import onnxruntime
n = onnxruntime.ModelMetadata

In [12]:
node = onnxruntime.NodeArg

In [14]:
node.name

In [2]:
import torch
import onnxruntime
import numpy
from collections import OrderedDict
ort_session = onnxruntime.InferenceSession("/data2/user/sjj995/TensorRT/temp2/vgg16-7.onnx",providers=['CPUExecutionProvider','CUDAExecutionProvider' ])

device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
x = torch.randn(1,3, 224,224).to(device)
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
outname = [output.name for output in ort_session.get_outputs()]
#print(ort_inputs)
#print(outname)
out_dict = OrderedDict()
ort_outs = ort_session.run(outname,ort_inputs)
for node, out in zip(ort_session.get_outputs(), ort_outs):
    out_dict[node.name] = out
# compare ONNX Runtime and PyTorch results
# np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

#print("Exported model has been tested with ONNXRuntime, and the result looks good!")
print((ort_outs))

[array([[-6.81014836e-01,  7.44424224e-01,  1.41540721e-01,
         5.63241541e-01,  1.69924617e+00,  7.35288441e-01,
         1.93122542e+00,  8.15252140e-02, -3.43908787e-01,
        -5.31984329e-01, -1.11749852e+00, -1.75256741e+00,
        -1.89881647e+00, -7.33805656e-01, -9.52086866e-01,
         8.49330425e-02, -1.94046080e+00, -1.10049796e+00,
         2.64512092e-01, -1.07474446e+00, -2.87014067e-01,
        -8.98062646e-01, -4.24095914e-02, -1.15856454e-02,
         9.61080417e-02,  4.87211227e-01, -1.03177047e+00,
        -3.24615538e-01, -1.41879618e-01, -1.70418695e-01,
        -3.56229484e-01,  8.45096409e-01, -2.66301364e-01,
         1.52067983e+00,  3.95152807e+00, -5.76735616e-01,
        -4.81660068e-01, -6.16991103e-01, -3.05021286e-01,
        -7.34090745e-01, -1.11675489e+00, -1.23026061e+00,
        -6.39251232e-01,  2.98177749e-01, -9.61876333e-01,
         1.11951518e+00, -4.58016396e-01, -8.53965044e-01,
         3.18903536e-01, -1.33759677e-02,  8.57594609e-

In [3]:
out_dict

OrderedDict([('vgg0_dense2_fwd',
              array([[-6.81014836e-01,  7.44424224e-01,  1.41540721e-01,
                       5.63241541e-01,  1.69924617e+00,  7.35288441e-01,
                       1.93122542e+00,  8.15252140e-02, -3.43908787e-01,
                      -5.31984329e-01, -1.11749852e+00, -1.75256741e+00,
                      -1.89881647e+00, -7.33805656e-01, -9.52086866e-01,
                       8.49330425e-02, -1.94046080e+00, -1.10049796e+00,
                       2.64512092e-01, -1.07474446e+00, -2.87014067e-01,
                      -8.98062646e-01, -4.24095914e-02, -1.15856454e-02,
                       9.61080417e-02,  4.87211227e-01, -1.03177047e+00,
                      -3.24615538e-01, -1.41879618e-01, -1.70418695e-01,
                      -3.56229484e-01,  8.45096409e-01, -2.66301364e-01,
                       1.52067983e+00,  3.95152807e+00, -5.76735616e-01,
                      -4.81660068e-01, -6.16991103e-01, -3.05021286e-01,
                  